In [1]:
import altair as alt
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

In [2]:
empath_data = pd.read_csv('Empath analysis.csv')
daily_covid_cases = pd.read_csv('Daily Covid Cases.csv')

In [3]:
empath_data['created_utc'] = pd.to_datetime(empath_data['created_utc'])
daily_covid_cases['Date']= pd.to_datetime(daily_covid_cases['Date'])
daily_covid_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    511 non-null    datetime64[ns]
 1   Count   511 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 8.1 KB


In [4]:
merge_data = empath_data.merge(daily_covid_cases, left_on='created_utc', right_on='Date', suffixes=(False, False))

In [5]:
merge_data.head()

,Unnamed: 0,subreddit,Unnamed: 0.1,subreddit_subscribers,title,id,author,created_utc,num_comments,score,...,monster,ocean,giving,contentment,writing,rural,positive_emotion,musical,Date,Count
0,0,singapore,1,378959,"About 71,600 in S'pore have psychotic disorder...",nhxjoa,MicrotechAnalysis,2021-05-22,36,117,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,2021-05-22,40
1,208,singapore,209,378960,A nowcast model of covid-19 spread in Singapore,ni3r47,isparavanje,2021-05-22,64,692,...,0.0,0.000884,0.001768,0.0,0.004421,0.0,0.006189,0.0,2021-05-22,40
2,264,SGExams,265,77716,[uni] is smu IS good? (Information system),nigjvj,Artistic_Sky4532,2021-05-22,23,19,...,0.0,0.000000,0.008772,0.0,0.000000,0.0,0.008772,0.0,2021-05-22,40
3,310,SGExams,311,77716,[A levels] Dunman High IP Programme - would yo...,nik80j,runawaypig44,2021-05-22,7,20,...,0.0,0.000000,0.010000,0.0,0.010000,0.0,0.000000,0.0,2021-05-22,40
4,358,SGExams,359,77717,[Uni] Internship & Overseas Exchange Chances/J...,nicahc,Awesome_Possum_0123,2021-05-22,16,5,...,0.0,0.000000,0.012500,0.0,0.006250,0.0,0.006250,0.0,2021-05-22,40


In [6]:
merge_data['previous_date'] = merge_data['Date'].shift()

#### Help Messages

In [155]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'help' : ['sum'],  
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'help', 'cases_count']

overall_negative_emotions['help'] = overall_negative_emotions['help'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

help_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'help'])
help  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['help'])

In [156]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(help_vs_covid, title='Help vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [158]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(help, title='Help Messages Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### Violence Messages

In [159]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'violence' : ['sum'],  
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'violence', 'cases_count']

overall_negative_emotions['violence'] = overall_negative_emotions['violence'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

violence_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'violence'])
violence  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['violence'])

In [160]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(violence_vs_covid, title='Violence vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [162]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'hate' : ['sum'],  
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'hate', 'cases_count']

overall_negative_emotions['hate'] = overall_negative_emotions['hate'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

hate_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'hate'])
hate  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['hate'])

### Hate Messages

In [163]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(hate_vs_covid, title='Hate vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [164]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(hate, title='Hate  Messages Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### Crime Messages

In [166]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'crime' : ['sum'],  
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'crime', 'cases_count']

overall_negative_emotions['crime'] = overall_negative_emotions['crime'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

crime_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'crime'])
crime  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['crime'])

In [167]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(crime_vs_covid, title='Crime vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [168]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(crime, title='Crime Messages Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### Envy

In [169]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'envy' : ['sum'],  
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'envy', 'cases_count']

overall_negative_emotions['envy'] = overall_negative_emotions['envy'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

envy_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'envy'])
envy  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['envy'])

In [170]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(envy_vs_covid, title='Envy vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [172]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(envy, title='Envy Messages Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [174]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'nervousness' : ['sum'],  
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'nervousness', 'cases_count']

overall_negative_emotions['nervousness'] = overall_negative_emotions['nervousness'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

nervousness_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'nervousness'])
nervousness  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['nervousness'])### Nervousness

In [175]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(nervousness_vs_covid, title='Nervousness vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [177]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(nervousness, title='Nervousness Messages Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### suffering

In [151]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'anger' : ['sum'], 
    'disappointment' : ['sum'], 
    'disgust' : ['sum'], 
    'dispute' : ['sum'], 
    'envy': ['sum'],
    'fear' : ['sum'], 
    'hate': ['sum'],
    'irritability' : ['sum'], 
    'kill': ['sum'],
    'nervousness': ['sum'],
    'suffering' : ['sum'], 
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'anger', 'disappointment', 'disgust', 
                                     'dispute','envy','fear', 'hate', 'irritability',  
                                     'kill', 'nervousness', 'suffering', 'cases_count']

overall_negative_emotions['suffering'] = overall_negative_emotions['suffering'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

suffering_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'suffering'])
suffering  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['suffering'])

In [152]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(suffering_vs_covid, title='Suffering vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [154]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(suffering, title='Suffering Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### Fear

In [146]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'anger' : ['sum'], 
    'disappointment' : ['sum'], 
    'disgust' : ['sum'], 
    'dispute' : ['sum'], 
    'envy': ['sum'],
    'fear' : ['sum'], 
    'hate': ['sum'],
    'irritability' : ['sum'], 
    'kill': ['sum'],
    'nervousness': ['sum'],
    'suffering' : ['sum'], 
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'anger', 'disappointment', 'disgust', 
                                     'dispute','envy','fear', 'hate', 'irritability',  
                                     'kill', 'nervousness', 'suffering', 'cases_count']

overall_negative_emotions['fear'] = overall_negative_emotions['fear'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

fear_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'fear'])
fear  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['fear'])

In [147]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(fear_vs_covid, title='Fear vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [148]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(fear, title='Fear Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### irritability

In [140]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'anger' : ['sum'], 
    'disappointment' : ['sum'], 
    'disgust' : ['sum'], 
    'dispute' : ['sum'], 
    'envy': ['sum'],
    'fear' : ['sum'], 
    'hate': ['sum'],
    'irritability' : ['sum'], 
    'kill': ['sum'],
    'nervousness': ['sum'],
    'suffering' : ['sum'], 
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'anger', 'disappointment', 'disgust', 
                                     'dispute','envy','fear', 'hate', 'irritability',  
                                     'kill', 'nervousness', 'suffering', 'cases_count']

overall_negative_emotions['irritability'] = overall_negative_emotions['irritability'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

irritability_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'irritability'])
irritability  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['irritability'])

In [141]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(irritability_vs_covid, title='Irritability vs Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [143]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(irritability, title='Irritability Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### Disappointment

In [132]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'anger' : ['sum'], 
    'disappointment' : ['sum'], 
    'disgust' : ['sum'], 
    'dispute' : ['sum'], 
    'envy': ['sum'],
    'fear' : ['sum'], 
    'hate': ['sum'],
    'irritability' : ['sum'], 
    'kill': ['sum'],
    'nervousness': ['sum'],
    'suffering' : ['sum'], 
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'anger', 'disappointment', 'disgust', 
                                     'dispute','envy','fear', 'hate', 'irritability',  
                                     'kill', 'nervousness', 'suffering', 'cases_count']

overall_negative_emotions['disappointment'] = overall_negative_emotions['disappointment'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

disappont_vs_covid = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'disappointment'])
disappont  = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['disappointment'])

In [124]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(disappont_vs_covid, title='Disappointment VS Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [144]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(disappont, title='Disappointment Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### Anger

In [134]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'anger' : ['sum'], 
    'disappointment' : ['sum'], 
    'disgust' : ['sum'], 
    'dispute' : ['sum'], 
    'envy': ['sum'],
    'fear' : ['sum'], 
    'hate': ['sum'],
    'irritability' : ['sum'], 
    'kill': ['sum'],
    'nervousness': ['sum'],
    'suffering' : ['sum'], 
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'anger', 'disappointment', 'disgust', 
                                     'dispute','envy','fear', 'hate', 'irritability',  
                                     'kill', 'nervousness', 'suffering', 'cases_count']

overall_negative_emotions['anger'] = overall_negative_emotions['anger'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions['daily_cases'].where(overall_negative_emotions['daily_cases'] > -100, -100, inplace=True)

anger_vs_covid_cases = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['daily_cases', 'anger'])
anger = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['anger'])

In [121]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(overall_negative_emotions_melt, title='Anger VS Covid Cases').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

In [145]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(anger, title='Anger Trend on Singapore During Pandemic').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)

### Overall Negative Emotions

In [129]:
overall_negative_emotions = merge_data.groupby(['created_utc']).agg({
    'anger' : ['sum'], 
    'disappointment' : ['sum'], 
    'disgust' : ['sum'], 
    'dispute' : ['sum'], 
    'envy': ['sum'],
    'fear' : ['sum'], 
    'hate': ['sum'],
    'irritability' : ['sum'], 
    'kill': ['sum'],
    'nervousness': ['sum'],
    'suffering' : ['sum'], 
    'Count': ['max']}).reset_index()

overall_negative_emotions.columns = ['created_utc', 'anger', 'disappointment', 'disgust', 
                                     'dispute','envy','fear', 'hate', 'irritability',  
                                     'kill', 'nervousness', 'suffering', 'cases_count']

overall_negative_emotions['anger'] = overall_negative_emotions['anger'] * 100
overall_negative_emotions['dispute'] = overall_negative_emotions['dispute'] * 100
overall_negative_emotions['disappointment'] = overall_negative_emotions['disappointment'] * 100
overall_negative_emotions['disgust'] = overall_negative_emotions['disgust'] * 100
overall_negative_emotions['dispute'] = overall_negative_emotions['dispute'] * 100
overall_negative_emotions['envy'] = overall_negative_emotions['envy'] * 100
overall_negative_emotions['fear'] = overall_negative_emotions['fear'] * 100

overall_negative_emotions['previous_case_count'] = overall_negative_emotions['cases_count'].shift()
overall_negative_emotions['cases_diff_percentage'] =  (overall_negative_emotions['cases_count'] -   overall_negative_emotions['previous_case_count'])
overall_negative_emotions = overall_negative_emotions.rename(columns={'cases_diff_percentage': 'daily_cases'})
overall_negative_emotions['daily_cases'] = overall_negative_emotions['daily_cases'] / overall_negative_emotions['cases_count'] *100

overall_negative_emotions[overall_negative_emotions['daily_cases'] < -100]['daily_cases'] = 100

overall_negative_emotions_melt = pd.melt(overall_negative_emotions, id_vars=['created_utc'], 
                                 value_vars= ['anger',  'disappointment',  'disgust', 'fear', 'hate', 
                                              'irritability', 'kill',
                                           'nervousness', 'suffering'])

<ipython-input-129-184d00f33905>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overall_negative_emotions[overall_negative_emotions['daily_cases'] < -100]['daily_cases'] = 100


In [131]:
selection = alt.selection_multi(fields=['variable'], bind='legend')

sentiment_trend = alt.Chart(overall_negative_emotions_melt, title='Overall Emotion Trend').mark_line().encode(
        x=alt.X(
            "created_utc:T",
            axis=alt.Axis(title="created_utc")
        ),
        y='value',
        color='variable:N',
    ).properties(
        width=800,
        height=400
    ).configure_axisX(
        labelAngle=-45,
    ).interactive().add_selection(
        selection
    )

sentiment_trend

alt.Chart(...)